# Analysis of Brazilian Development Indicators

In [84]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Importing database
original_df = pd.read_csv('Data.csv')

# Transposing database for setting features as columns
df_transpose = original_df.T

# Selecting rows and columns that are missing to many entries
rows_to_drop = ['Country Name', 'Country Code', 'Series Code']
columns_to_drop = [4,14,17,19,25,28,30,41,55,56,57,58,59]

# Dropping selected rows and columns
df_dropped_rows = df_transpose.drop(rows_to_drop)
df_dropped_columns_and_rows = df_dropped_rows.drop(columns_to_drop,axis=1)

# Substituting the remaining missing entries by NaN
df_smaller = df_dropped_columns_and_rows.replace('..', np.nan)

# Dropping header for imputation and analysis
df_headless = df_smaller.drop('Series Name')

# Imputing NaN using the mean strategy
imp_mean = SimpleImputer(strategy='mean')
df = pd.DataFrame(imp_mean.fit_transform(df_headless))

# Checking the most important quantities
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
count,1.200000e+01,12.000000,12.0,12.000000,12.000000,1.200000e+01,12.000000,1.200000e+01,12.00000,12.000000,...,12.000000,12.000000,12.000000,12.000000,1.200000e+01,12.000000,12.000000,1.200000e+01,1.200000e+01,1.200000e+01
mean,2.014250e+08,0.880462,8515770.0,23.778737,6.330000,1.736959e+12,8446.666667,2.829747e+12,13795.00000,3.350000,...,13.144457,78.055551,21.488646,109.252277,4.832103e+11,42.410489,38457.727273,2.867352e+09,6.065763e+10,6.264173e+08
std,1.908090e+07,0.367124,0.0,2.156543,5.719035,6.290065e+11,2884.807645,7.782785e+11,3068.93378,0.497722,...,1.836142,3.723009,5.903429,15.064965,1.471823e+11,17.330506,28150.646959,1.137732e+09,2.627712e+10,3.521998e+08
min,1.507064e+08,0.459584,8515770.0,18.031099,1.900000,3.896922e+11,2590.000000,9.708541e+11,6440.00000,2.300000,...,9.001535,72.222233,11.599412,66.442953,1.203119e+11,22.638332,-1761.000000,5.730000e+08,9.890000e+08,1.743800e+08
25%,2.030252e+08,0.740875,8515770.0,23.778737,3.850000,1.691209e+12,7907.500000,2.937705e+12,14135.00000,3.175000,...,12.270721,75.555556,18.363595,108.776068,4.836037e+11,31.875530,16896.250000,2.686055e+09,4.427588e+10,2.735425e+08
50%,2.076823e+08,0.802512,8515770.0,24.446100,5.300000,1.824244e+12,8780.000000,3.074636e+12,14570.00000,3.325000,...,13.144457,78.055551,19.991748,112.346801,5.433273e+11,41.086300,36882.863636,2.818348e+09,6.902995e+10,6.194737e+08
75%,2.121362e+08,0.860463,8515770.0,24.996036,5.932500,1.985862e+12,9455.000000,3.134553e+12,15332.50000,3.525000,...,14.355218,78.819439,22.977458,115.770594,5.571254e+11,50.358831,59643.500000,3.301771e+09,7.594895e+10,9.364100e+08
max,2.153135e+08,1.809856,8515770.0,25.507625,24.000000,2.571102e+12,12750.000000,3.716656e+12,17260.00000,4.500000,...,15.999958,84.444433,32.622387,125.759743,6.064846e+11,86.182161,82589.000000,4.969231e+09,9.150210e+10,1.155730e+09


In [85]:
# Setting the target for prediction
y = df.loc[:,32] # target is GDP Growth (annual %)

In [86]:
# Defining the correlation matrix
df_corr = df.corr()

# Transforming the correlation matrix into an upper triangular matrix
upper_df_corr = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))

# Selecting columns that present at least one entry of correlation greater than 0.95
high_corr_columns = [column for column in upper_df_corr.columns if any(upper_df_corr[column] > 0.95)]

# Defining the features to be used based on the previously selected columns
X = df.loc[:,high_corr_columns]

# Checking features to be used
X.head()

,6,7,8,10,14,20,21,22,24,26,28,35,36,39,40,41
0,2590.0,9.708541e+11,6440.0,65.985,63.2,5888980.0,0.735736,2.874730,4.649891e+11,2736.970678,33.271883,2.363841,0.000443,11.599412,66.442953,1.203119e+11
1,3910.0,1.536382e+12,8740.0,69.737,34.7,5510886.0,0.991168,2.227694,6.554482e+11,5.606065,23.006619,1.730726,13.184557,17.356368,100.000000,2.425117e+11
2,12750.0,3.090348e+12,15320.0,73.918,16.7,5069631.6,1.190697,1.204556,2.472819e+12,7.504565,21.224810,1.329446,134.392933,19.606204,125.759743,4.837348e+11
3,12080.0,3.127765e+12,15370.0,74.306,16.3,5054239.8,1.148596,1.189409,2.456044e+12,7.846710,20.472463,1.330244,137.977627,18.797955,121.483706,5.569196e+11
4,10160.0,2.958026e+12,14420.0,74.332,15.9,5038848.0,1.128069,1.170641,1.802212e+12,7.566175,19.360044,1.365517,125.647707,20.377292,107.347442,5.433974e+11


In [104]:
# Splitting dataframe for training and validation of the model
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# Applying Random Forest model
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)

# Checking the model's MAE
y_prediction = forest_model.predict(val_X)
print("The model's mean absolute error is:")
print(mean_absolute_error(val_y, y_prediction))

# Checking model's predictions
prediction = forest_model.predict(X)
difference = y-prediction
print("The predictions are:")
print(prediction)

# Checking prediction's standard deviation
print("The standard deviation associated to the predicted values is:")
print(np.std(prediction))

# Checking mean difference between true and predicted values
print("The mean difference between the actual values and predicted values is:")
print(difference.mean())


The model's mean absolute error is:
0.6538899120975662
The predictions are:
[28.18969306 28.15879452 28.19917786 27.8097134  28.52904271 29.28317788
 28.50493314 28.67445285 28.36574661 26.41237762 27.56616526 27.64957419]
The standard deviation associated to the predicted values is:
0.6824228691778765
The mean difference between the actual values and predicted values is:
0.09674765638843397


# Conclusions

In this work, a Random Forest regression was applied to a dataset of Brazilian development indicators through the years 1990 to 2022 in order to predict Brazil's GDP. The result was a model of 0.65 mean average error, what implies the model was successful in its predictions. Note, however, that in order for this model to be capable of predicting future values for GDP, it would need to be fed with data from other development indicators of the year it is intended to predict.